In [1]:
import GEOparse
import pandas as pd

def download_experiment(experiment):
    GEOparse.get_GEO(geo=experiment, destdir="data/")

def read_experiment(experiment):
    return GEOparse.get_GEO(filepath="data/" + experiment + "_family.soft.gz")

def aggregate_samples(experiment):
    # download_experiment(experiment)
    print('reading experiment...')
    gse = read_experiment(experiment)
    aggregated_samples = pd.DataFrame()
    genes = pd.DataFrame()
    print('aggregating samples...')
    for sample in gse.gsms:
        name = gse.gsms[sample].metadata['title'][0]
        sample_df = gse.gsms[sample].table
        sample_df.columns = ['gene', name]
        genes = sample_df['gene']
        aggregated_samples = pd.concat([aggregated_samples, sample_df[name]], axis=1)    
    aggregated_samples = pd.concat([genes, aggregated_samples], axis=1)
    aggregated_samples.to_csv('data/' + experiment + '_full_matrix.csv') 
    print('aggregated samples written to csv file')

eichenberger = 'GSE67023'
# aggregate_samples(eichenberger)


In [94]:
from collections import Counter, defaultdict
from scipy.stats import multivariate_normal
from sklearn.preprocessing import normalize
import numpy as np
import random

print('reading data...')
df = pd.read_csv('data/eichenberger_full_matrix.csv').fillna(0)
gene_data = normalize(df.values[:, 2:].astype(float), axis=0, norm='l1')
sample_data = normalize(df.values[:, 2:].astype(float), axis=0, norm='l1').transpose()

i2s = list(df)[2:] # index to sample name
s2i = {sample: i for i, sample in enumerate(i2s)} # sample name to index

i2g = df.values[:, 1] # index to gene name
g2i = {gene: i for i, gene in enumerate(i2g)} # gene name to index

def distribution_parameters(gene, genes):
    gene_differences = np.array([g-gene for g in genes])
    covariance = np.cov(gene_differences, rowvar=False)
    mean = np.mean(gene_differences, axis=0)
    return mean, covariance

def score_multivariate(x, mean, covariance):
    return 1-multivariate_normal.pdf(x, mean=mean, cov=covariance)

def score_cosine(x, y):
    return (np.dot(x, np.transpose(y))/(np.linalg.norm(x)*np.linalg.norm(y)))

# def sample_similarities(similarity_function=gene_similarity_cosine):
    
def sequential_similarities(data, similarity_function=score_cosine):
    print("calculating sequential similarity...")
    similarities = [similarity_function(data[i], data[i+1]) for i in range(0, len(data)-1)]
    return similarities

def similarities(data, similarity_function=score_cosine):
    print("calculating all similarities...")
    similarities = []
    for x in data:       
        similarities.append([similarity_function(x, y) for y in data])
    return similarities

gene_similarities = sequential_similarities(gene_data)
# multivariate_similarities = sequential_similarities[]
sample_similarities_cosine = similarities(sample_data)

def most_similar_samples(sample, sample_similarities, n=5):
    print('Finding most similar samples for', sample + '...')
    s = sample_similarities[s2i[sample]]
    best = reversed(np.argsort(s)[-n:])
    return [i2s[b] for b in best]

random_sample = i2s[random.randint(0, len(i2s))]
print(most_similar_samples(random_sample, sample_similarities_cosine))

def threshold_operons_sequential(similarities, threshold=0.6):
    '''threshold a sequential list of similarities. 
    This means the spatial location of the genes is used
    '''
    cluster = 0
    threshold_operons = defaultdict(list)
    print('thresholding similarity into operons...')
    for i, sim in enumerate(similarities):
        if sim > threshold:
            threshold_operons[cluster].append(i2g[i])
        else:
            cluster += 1
            threshold_operons[cluster].append(i2g[i])
    print('done.')
    return threshold_operons
    
operons = threshold_operons_sequential(gene_similarities)

reading data...
calculating sequential similarity...
calculating similarities...
Finding most similar samples for YoaUT3.5...
['YoaUT3.5', 'YoaUT5.5', 'Rifamp1_T15', 'Gin(6825vs6827)2_T3', 'Vanco2_T0']
thresholding similarity into operons...
done.


In [3]:
from sklearn.cluster import KMeans, AffinityPropagation
# AP = AffinityPropagation(damping = 0.5, max_iter = 250, affinity = 'euclidean').fit(cluster_data)
# clusters = AP.fit_predict(cluster_data)
Counter(clusters)
names_per_cluster = defaultdict(list)
for i, label in enumerate(clusters):
    names_per_cluster[label].append(i2g[i])
# names_per_cluster = {label: names.append(i2g[i]) for i, label in enumerate(clusters)}
# names_per_cluster
names_per_cluster

In [95]:
import csv
import json

def read_operons(path):
    operons_per_id = defaultdict(list)
    with open(path, 'r') as infile:
        reader = csv.reader(infile, delimiter='\t')
        header = next(reader)
        for line in reader:
            operon_id = line[0]
            gene_id = line[2]
            operons_per_id[operon_id].append(gene_id)
    
    operons_per_gene = defaultdict(list)
    for operon_list in operons_per_id.values():
        for gene in operon_list:
            operons_per_gene[gene] = operon_list
    return operons_per_gene

true_operons = json.load(open('data/operons.json'))

def validate_operon(gene, predicted_operon):
    '''
    Validates the operon (set) of a gene (id string)
    on recall and precision of the operon members
    using the gold standard operon dict
    '''
    try:
        true_operon = set(true_operons[gene])
    except:
#         print("Gene not found in gold standard set")
        return
#     print("This gene is part of the following (true) operon:", true_operon)
#     print("The predicted operon for this gene is:", set(predicted_operon))
    recall = len(true_operon.intersection(predicted_operon))/len(true_operon)
#     print("The recall of the predicted operon is:", recall)
    precision = len(true_operon.intersection(predicted_operon))/len(predicted_operon)
#     print("The precision of the predicted operon is:", precision)
    correct = int(set(predicted_operon) == true_operon)
    return recall, precision, correct
       
    
def validate_operons(dictionary):
    recall_scores = []
    precision_scores = []
    predictions = []
    print('validating predicted operons...')
    for cluster in dictionary.values():
        for gene in cluster:
            score = validate_operon(gene, cluster)
            if score:
                recall_scores.append(score[0])
                precision_scores.append(score[1])
                predictions.append(score[2])
            else:
                continue
    mean_recall = np.mean(recall_scores)
    mean_precision = np.mean(precision_scores)
    accuracy = np.mean(predictions)
    return mean_recall, mean_precision, accuracy

benchmark = {gene: [gene] for gene in i2g}

r, p, a = validate_operons(benchmark)
print("The mean recall of the benchmark is:", r)
print("The mean precision of the benchmark is:", p)
print("The accuracy of the benchmark is:", a)
r, p, a = validate_operons(operons)

print("The mean recall of the assigned operons is:", r)
print("The mean precision of the assigned operons is:", p)
print("The accuracy of the assigned operons is:", a)


# def get_operon(gene):
    



validating predicted operons...
The mean recall of the benchmark is: 0.6034474017743979
The mean precision of the benchmark is: 1.0
The accuracy of the benchmark is: 0.40811153358681873
validating predicted operons...
The mean recall of the assigned operons is: 0.7676228883072989
The mean precision of the assigned operons is: 0.7498756569870776
The accuracy of the assigned operons is: 0.26134347275031683
